# 🔍 Consultas CRUD - Proyecto MongoDB Replicación

## 🎯 Objetivos
- Realizar 15 consultas CRUD complejas
- Probar operaciones de lectura y escritura
- Verificar consistencia en replicación
- Analizar rendimiento de consultas

---

## 1. 📦 Importar Librerías y Conectar

In [10]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Conectar a MongoDB
client = MongoClient('mongodb://localhost:27020/', directConnection=True)
db = client['ecommerce_brazil']
collection = db['ventas']

print("✅ Conectado a MongoDB")
print(f"📊 Documentos en colección: {collection.count_documents({}):,}")

# Verificar estructura de un documento
doc_ejemplo = collection.find_one()
print("\n📋 Estructura de documento:")
for key, value in doc_ejemplo.items():
    print(f"  {key}: {type(value).__name__}")

✅ Conectado a MongoDB
📊 Documentos en colección: 118,310

📋 Estructura de documento:
  _id: ObjectId
  id_orden: str
  id_item: int
  id_producto: str
  id_vendedor: str
  shipping_limit_date: str
  precio: float
  costo_envio: float
  id_cliente: str
  estado_orden: str
  fecha_compra: datetime
  order_approved_at: str
  order_delivered_carrier_date: str
  order_delivered_customer_date: str
  order_estimated_delivery_date: str
  id_cliente_unico: str
  customer_zip_code_prefix: int
  ciudad_cliente: str
  estado_cliente: str
  product_category_name: str
  longitud_nombre: float
  longitud_descripcion: float
  cantidad_fotos: float
  peso_gramos: float
  longitud_cm: float
  altura_cm: float
  ancho_cm: float
  categoria_producto: str
  seller_zip_code_prefix: int
  ciudad_vendedor: str
  estado_vendedor: str
  payment_sequential: float
  tipo_pago: str
  cuotas_pago: float
  payment_value: float
  review_id: str
  puntuacion_review: float
  review_comment_title: float
  comentario_rev

## 2. 🔍 Consulta 1: Ventas de los últimos 3 meses por cliente

In [19]:
# Consulta 1: Ventas de los últimos 3 meses para un cliente específico
print("🔍 Consulta 1: Ventas de los últimos 3 meses por cliente")

# Fecha de hace 3 meses
# Fecha de hace 3 meses (ajustada para el dataset)
fecha_limite = "2018-02-01"  # 3 meses antes del final del dataset (2018-05)
fecha_limite_str = fecha_limite

# Cliente específico (usando el primer cliente único disponible)
cliente_ejemplo = collection.find_one({}, {'id_cliente_unico':3})['id_cliente_unico']

query = {
    'id_cliente_unico': cliente_ejemplo,
    'fecha_compra': {'$gte': fecha_limite_str}
}

result = list(collection.find(query).sort('fecha_compra', -1))
print(f"📊 Ventas para cliente {cliente_ejemplo} en los últimos 3 meses: {len(result)}")
if result:
    df_result = pd.DataFrame(result)
    print(df_result[['id_orden', 'fecha_compra', 'precio_total', 'categoria_producto']].head())

🔍 Consulta 1: Ventas de los últimos 3 meses por cliente
📊 Ventas para cliente 871766c5855e863f6eccc05f988b23cb en los últimos 3 meses: 0


## 3. 🔍 Consulta 2: Total gastado por cliente agrupado por producto

In [4]:
# Consulta 2: Total gastado por cliente agrupado por producto
print("🔍 Consulta 2: Total gastado por cliente agrupado por producto")

pipeline = [
    {
        '$match': {
            'id_cliente_unico': cliente_ejemplo,
            'fecha_compra': {'$gte': fecha_limite_str}
        }
    },
    {
        '$group': {
            '_id': '$categoria_producto',
            'total_gastado': {'$sum': '$precio_total'},
            'cantidad_compras': {'$sum': 1}
        }
    },
    {'$sort': {'total_gastado': -1}}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print(df_result.head(10))

🔍 Consulta 2: Total gastado por cliente agrupado por producto
Empty DataFrame
Columns: []
Index: []


## 4. 🔍 Consulta 3: Productos con stock disminuido más del 15%

In [5]:
# Consulta 3: Productos con stock disminuido más del 15%
print("🔍 Consulta 3: Productos con stock disminuido más del 15%")

# Fechas para mes actual y anterior
mes_actual = "2017-12-01"  # Mes actual del dataset
mes_anterior = "2017-11-01"  # Mes anterior del dataset

pipeline = [
    {
        '$group': {
            '_id': '$id_producto',
            'stock_actual': {'$avg': '$cantidad_stock'},
            'compras_mes_actual': {
                '$sum': {
                    '$cond': [
                        {'$gte': ['$fecha_compra', mes_actual]},
                        1, 0
                    ]
                }
            },
            'compras_mes_anterior': {
                '$sum': {
                    '$cond': [
                        {'$gte': ['$fecha_compra', mes_anterior]},
                        1, 0
                    ]
                }
            }
        }
    },
    {
        '$addFields': {
            'disminucion_stock': {
                '$multiply': [
                    {'$divide': [
                        {'$subtract': ['$compras_mes_anterior', '$compras_mes_actual']},
                        '$compras_mes_anterior'
                    ]},
                    100
                ]
            }
        }
    },
    {'$match': {'disminucion_stock': {'$gt': 15}}},
    {'$sort': {'disminucion_stock': -1}}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print(f"📊 Productos con stock disminuido >15%: {len(df_result)}")
if len(df_result) > 0:
    print(df_result.head())

🔍 Consulta 3: Productos con stock disminuido más del 15%


OperationFailure: PlanExecutor error during aggregation :: caused by :: can't $divide by zero, full error: {'ok': 0.0, 'errmsg': "PlanExecutor error during aggregation :: caused by :: can't $divide by zero", 'code': 2, 'codeName': 'BadValue', '$clusterTime': {'clusterTime': Timestamp(1754525752, 1), 'signature': {'hash': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 'keyId': 0}}, 'operationTime': Timestamp(1754525752, 1)}

## 5. 🔍 Consulta 4: Productos por ciudad con precio superior al promedio

In [ ]:
# Consulta 4: Productos por ciudad con precio superior al promedio
print("🔍 Consulta 4: Productos por ciudad con precio superior al promedio")

# Ciudad específica
ciudad_ejemplo = collection.find_one({}, {'ciudad_cliente': 1})['ciudad_cliente']

pipeline = [
    {'$match': {'ciudad_cliente': ciudad_ejemplo}},
    {
        '$group': {
            '_id': None,
            'promedio_precio': {'$avg': '$precio_total'}
        }
    }
]

promedio = list(collection.aggregate(pipeline))[0]['promedio_precio']

query = {
    'ciudad_cliente': ciudad_ejemplo,
    'precio_total': {'$gt': promedio}
}

result = list(collection.find(query).limit(10))
df_result = pd.DataFrame(result)
print(f"📊 Productos en {ciudad_ejemplo} con precio > ${promedio:.2f}: {len(result)}")
if len(df_result) > 0:
    print(df_result[['id_producto', 'precio_total', 'categoria_producto']].head())

## 6. 🔍 Consulta 5: Actualizar precios en rango de fechas

In [ ]:
# Consulta 5: Actualizar precios en rango de fechas (simulación)
print("🔍 Consulta 5: Actualizar precios en rango de fechas (simulación)")

# Rango de fechas específico
fecha_inicio = '2017-01-01'  # Inicio del dataset
fecha_fin = '2017-03-31'  # Primer trimestre del dataset

# Contar documentos que se actualizarían
query = {
    'fecha_compra': {'$gte': fecha_inicio, '$lte': fecha_fin},
    'cantidad_stock': {'$gt': 10}
}

count = collection.count_documents(query)
print(f"📊 Documentos que se actualizarían: {count}")

# Simular la actualización (no ejecutar en producción)
print("💡 Para ejecutar la actualización real, usar:")
print("""
result = collection.update_many(
    query,
    {'$mul': {'precio': 1.10}}
)
""")

## 7. 🔍 Consulta 6: Actualizar email de cliente con condiciones

In [ ]:
# Consulta 6: Actualizar email de cliente con condiciones (simulación)
print("🔍 Consulta 6: Actualizar email de cliente con condiciones")

# Fecha del último trimestre
ultimo_trimestre = "2017-09-01"  # Último trimestre del dataset

pipeline = [
    {
        '$group': {
            '_id': '$id_cliente_unico',
            'total_compras': {'$sum': 1},
            'ultima_compra': {'$max': '$fecha_compra'}
        }
    },
    {
        '$match': {
            '$and': [
                {'total_compras': {'$gt': 5}},
                {'ultima_compra': {'$gte': ultimo_trimestre}}
            ]
        }
    }
]

result = list(collection.aggregate(pipeline))
print(f"📊 Clientes que cumplen condiciones: {len(result)}")
if len(result) > 0:
    df_result = pd.DataFrame(result)
    print(df_result.head())

## 8. 🔍 Consulta 7: Actualizar precios de productos vendidos más de 100 veces

In [ ]:
# Consulta 7: Productos vendidos más de 100 veces en el último año
print("🔍 Consulta 7: Productos vendidos más de 100 veces en el último año")

fecha_ultimo_anio = "2017-01-01"  # Inicio del año del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_anio}
        }
    },
    {
        '$group': {
            '_id': '$id_producto',
            'total_ventas': {'$sum': 1},
            'precio_promedio': {'$avg': '$precio'}
        }
    },
    {
        '$match': {
            '$and': [
                {'total_ventas': {'$gt': 100}},
                {'precio_promedio': {'$lt': 100}}
            ]
        }
    },
    {'$sort': {'total_ventas': -1}}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print(f"📊 Productos que cumplen condiciones: {len(df_result)}")
if len(df_result) > 0:
    print(df_result.head())

## 9. 🔍 Consulta 8: Eliminar productos con stock 0 y sin ventas recientes

In [ ]:
# Consulta 8: Productos con stock 0 y sin ventas en 6 meses (simulación)
print("🔍 Consulta 8: Productos con stock 0 y sin ventas en 6 meses")

fecha_6_meses = "2017-06-01"  # 6 meses antes del final del dataset

pipeline = [
    {
        '$group': {
            '_id': '$id_producto',
            'stock_actual': {'$avg': '$cantidad_stock'},
            'ultima_venta': {'$max': '$fecha_compra'}
        }
    },
    {
        '$match': {
            '$and': [
                {'stock_actual': 0},
                {'ultima_venta': {'$lt': fecha_6_meses}}
            ]
        }
    }
]

result = list(collection.aggregate(pipeline))
print(f"📊 Productos que se eliminarían: {len(result)}")
if len(result) > 0:
    df_result = pd.DataFrame(result)
    print(df_result.head())

## 10. 🔍 Consulta 9: Eliminar ventas por ciudad con precio bajo promedio

In [ ]:
# Consulta 9: Ventas por ciudad con precio bajo promedio (simulación)
print("🔍 Consulta 9: Ventas por ciudad con precio bajo promedio")

fecha_ultimo_trimestre = "2017-09-01"  # Último trimestre del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_trimestre}
        }
    },
    {
        '$group': {
            '_id': '$ciudad_cliente',
            'promedio_precio': {'$avg': '$precio_total'}
        }
    }
]

promedios_ciudad = list(collection.aggregate(pipeline))
print(f"📊 Ciudades analizadas: {len(promedios_ciudad)}")

# Simular eliminación para una ciudad
if promedios_ciudad:
    ciudad = promedios_ciudad[0]['_id']
    promedio = promedios_ciudad[0]['promedio_precio']
    
    query = {
        'ciudad_cliente': ciudad,
        'precio_total': {'$lt': promedio}
    }
    
    count = collection.count_documents(query)
    print(f"📊 Ventas en {ciudad} que se eliminarían: {count}")

## 11. 🔍 Consulta 10: Eliminar clientes con compras bajas

In [ ]:
# Consulta 10: Clientes con compras totales < $100 en el último año
print("🔍 Consulta 10: Clientes con compras totales < $100 en el último año")

fecha_ultimo_anio = "2017-01-01"  # Inicio del año del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_anio}
        }
    },
    {
        '$group': {
            '_id': '$id_cliente_unico',
            'total_compras': {'$sum': '$precio_total'}
        }
    },
    {'$match': {'total_compras': {'$lt': 100}}},
    {'$sort': {'total_compras': 1}}
]

result = list(collection.aggregate(pipeline))
print(f"📊 Clientes con compras < $100: {len(result)}")
if len(result) > 0:
    df_result = pd.DataFrame(result)
    print(df_result.head())

## 12. 🔍 Consulta 11: Total de ventas por cliente en el último año

In [ ]:
# Consulta 11: Total de ventas por cliente en el último año
print("🔍 Consulta 11: Total de ventas por cliente en el último año")

fecha_ultimo_anio = "2017-01-01"  # Inicio del año del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_anio}
        }
    },
    {
        '$group': {
            '_id': '$id_cliente_unico',
            'total_ventas': {'$sum': '$precio_total'},
            'cantidad_compras': {'$sum': 1},
            'promedio_precio': {'$avg': '$precio_total'}
        }
    },
    {
        '$project': {
            'cliente_id': '$_id',
            'total_ventas': 1,
            'cantidad_compras': 1,
            'promedio_precio': {'$round': ['$promedio_precio', 2]}
        }
    },
    {'$sort': {'total_ventas': -1}},
    {'$limit': 10}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print("📊 Top 10 clientes por ventas totales:")
print(df_result)

## 13. 🔍 Consulta 12: Productos más vendidos en el último trimestre

In [ ]:
# Consulta 12: Productos más vendidos en el último trimestre
print("🔍 Consulta 12: Productos más vendidos en el último trimestre")

fecha_ultimo_trimestre = "2017-09-01"  # Último trimestre del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_trimestre}
        }
    },
    {
        '$group': {
            '_id': '$categoria_producto',
            'cantidad_vendida': {'$sum': 1},
            'ingresos_totales': {'$sum': '$precio_total'}
        }
    },
    {
        '$project': {
            'producto': '$_id',
            'cantidad_vendida': 1,
            'ingresos_totales': {'$round': ['$ingresos_totales', 2]}
        }
    },
    {'$sort': {'cantidad_vendida': -1}},
    {'$limit': 10}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print("📊 Top 10 productos más vendidos:")
print(df_result)

## 14. 🔍 Consulta 13: Total de ventas por ciudad en el último mes

In [ ]:
# Consulta 13: Total de ventas por ciudad en el último mes
print("🔍 Consulta 13: Total de ventas por ciudad en el último mes")

fecha_ultimo_mes = "2017-11-01"  # Último mes del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_mes}
        }
    },
    {
        '$group': {
            '_id': '$ciudad_cliente',
            'total_ventas': {'$sum': '$precio_total'},
            'cantidad_ventas': {'$sum': 1}
        }
    },
    {
        '$project': {
            'ciudad': '$_id',
            'total_ventas': {'$round': ['$total_ventas', 2]},
            'cantidad_ventas': 1
        }
    },
    {'$sort': {'total_ventas': -1}},
    {'$limit': 15}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print("📊 Top 15 ciudades por ventas totales:")
print(df_result)

## 15. 🔍 Consulta 14: Correlación entre precio y stock

In [ ]:
# Consulta 14: Correlación entre precio y stock
print("🔍 Consulta 14: Correlación entre precio y stock")

pipeline = [
    {
        '$group': {
            '_id': '$id_producto',
            'precio_promedio': {'$avg': '$precio'},
            'stock_promedio': {'$avg': '$cantidad_stock'}
        }
    },
    {
        '$project': {
            'producto_id': '$_id',
            'precio_promedio': {'$round': ['$precio_promedio', 2]},
            'stock_promedio': {'$round': ['$stock_promedio', 2]}
        }
    },
    {'$limit': 1000}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)

# Calcular correlación
correlacion = df_result['precio_promedio'].corr(df_result['stock_promedio'])
print(f"📊 Correlación entre precio y stock: {correlacion:.4f}")
print(f"📊 Muestra analizada: {len(df_result)} productos")

# Visualización
plt.figure(figsize=(10, 6))
plt.scatter(df_result['precio_promedio'], df_result['stock_promedio'], alpha=0.6)
plt.xlabel('Precio Promedio')
plt.ylabel('Stock Promedio')
plt.title(f'Correlación Precio vs Stock (r = {correlacion:.4f})')
plt.grid(True, alpha=0.3)
plt.show()

## 16. 🔍 Consulta 15: Top 5 productos con mayor ventas (excluyendo stock bajo)

In [ ]:
# Consulta 15: Top 5 productos con mayor ventas (excluyendo stock bajo)
print("🔍 Consulta 15: Top 5 productos con mayor ventas (excluyendo stock bajo)")

fecha_ultimo_trimestre = "2017-09-01"  # Último trimestre del dataset

pipeline = [
    {
        '$match': {
            'fecha_compra': {'$gte': fecha_ultimo_trimestre},
            'cantidad_stock': {'$gte': 10}
        }
    },
    {
        '$group': {
            '_id': '$categoria_producto',
            'cantidad_vendida': {'$sum': 1},
            'ingresos_totales': {'$sum': '$precio_total'}
        }
    },
    {
        '$project': {
            'producto': '$_id',
            'cantidad_vendida': 1,
            'ingresos_totales': {'$round': ['$ingresos_totales', 2]}
        }
    },
    {'$sort': {'cantidad_vendida': -1}},
    {'$limit': 5}
]

result = list(collection.aggregate(pipeline))
df_result = pd.DataFrame(result)
print("📊 Top 5 productos más vendidos (stock >= 10):")
print(df_result)

# Visualización
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.bar(df_result['producto'], df_result['cantidad_vendida'])
plt.title('Cantidad Vendida')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(df_result['producto'], df_result['ingresos_totales'])
plt.title('Ingresos Totales')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 17. 🔄 Verificar Replicación en Consultas CRUD

In [ ]:
# Verificar que las consultas funcionan en nodos secundarios
print("🔄 Verificando replicación en consultas CRUD...")

try:
    # Conectar a nodos secundarios
    from pymongo import MongoClient
    
    # Secundario 1
    secondary1_client = MongoClient('mongodb://localhost:27021/', directConnection=True)
    secondary1_db = secondary1_client['ecommerce_brazil']
    secondary1_collection = secondary1_db['ventas']
    
    # Secundario 2
    secondary2_client = MongoClient('mongodb://localhost:27022/', directConnection=True)
    secondary2_db = secondary2_client['ecommerce_brazil']
    secondary2_collection = secondary2_db['ventas']
    
    # Verificar conteo en todos los nodos
    primary_count = collection.count_documents({})
    secondary1_count = secondary1_collection.count_documents({})
    secondary2_count = secondary2_collection.count_documents({})
    
    print(f"✅ Primario: {primary_count:,} documentos")
    print(f"✅ Secundario 1: {secondary1_count:,} documentos")
    print(f"✅ Secundario 2: {secondary2_count:,} documentos")
    
    if primary_count == secondary1_count == secondary2_count:
        print("🎉 ¡Replicación perfecta en consultas CRUD!")
    else:
        print("⚠️ Diferencia en replicación")
        
    # Cerrar conexiones
    secondary1_client.close()
    secondary2_client.close()
    
except Exception as e:
    print(f"❌ Error verificando replicación: {e}")

In [ ]:
# Cerrar conexión principal
client.close()
print("🔌 Conexión cerrada")
print("\n🎉 ¡Todas las consultas CRUD completadas exitosamente!")